# Preprocessing

We want to make all bounding boxes detected by MegaDetector into point-level annotations.


In [2]:
from config import *
import pandas as pd
import json

In [3]:
with open(MEGATRON_BOUNDING_BOXES, encoding="utf-8") as f:
    megadetector_results = json.load(f)
megadetector_results.keys()

dict_keys(['info', 'images', 'detection_categories'])

In [4]:
# dataframe for bbox
megadetector_results_df = pd.DataFrame(megadetector_results["images"])

In [5]:
# bounding boxes
bb = [detection['bbox'] for detection in megadetector_results_df.iloc[175]["detections"]]

In [6]:
# helper method that converts a list of bounding boxes to a list of point-level annotations
def bb_to_point(bb, im):
    img_width, img_height = im.size[0], im.size[1]
    return [(round(((2*box[0] + box[2])/2) * img_width),round(((2*box[1] + box[3])/2) * img_height)) for box in bb]

# Binary count using LC-FCN

In [7]:
from lcfcn import lcfcn, lcfcn_loss
from PIL import Image
import torch

In [8]:
# selecting an image with its associated bounding boxes
data_index = 175

# Load 100th image data. 
im = Image.open(TRAIN_DATA_DIR + "/" + megadetector_results_df.loc[data_index]['id'] + ".jpg")

In [9]:
batch = {}
batch['images'] = lcfcn.transform_image(im)[None]
batch['points'] = torch.zeros(batch['images'].shape[-2:])[None]
point_list = points = bb_to_point(bb,im)

In [10]:
point_list

[(776, 351),
 (641, 348),
 (489, 329),
 (192, 321),
 (276, 310),
 (56, 311),
 (225, 312),
 (161, 315),
 (830, 249)]

In [ ]:
# Train and Validate
model = lcfcn.LCFCN(n_classes=1, lr=1e-5, device='cpu')

for y,x in point_list:
  batch['points'][:, y, x] = 1

# train for several iterations
for i in range(1000):
    loss = model.train_on_batch(batch)
    val_dict = model.val_on_batch(batch)
    print(i, '- loss:', float(loss['train_loss']), val_dict)
    if i % 10 == 0:
        # visualize blobs and heatmap
        model.vis_on_batch(batch, savedir_image='result.png')

0 - loss: 13.169797897338867 {'mae': 104.0, 'game': 120.0}
